## Imports

In [1]:
import numpy as np 
import pandas as pd 
import os
import gc
import copy
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.applications.resnet50 import preprocess_input, decode_predictions

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

ValueError: Unable to import backend : plaidml.keras.backend

## Set Data path

In [ ]:
datadir = os.path.expanduser('~/Dev/humpbackwhale/data')
pretrainedmodeldir = os.path.expanduser('~/Dev/humpbackwhale/pretrainedmodels')
os.listdir(os.path.expanduser(datadir))

In [ ]:
train_df = pd.read_csv(datadir + '/train.csv')
#train_df = train_df[:500]
train_df.head()

In [ ]:
def prepareImages(data, m, dataset):
    print("Preparing images")
    X_train = np.zeros((m, 100, 100, 3))
    count = 0
    
    for fig in data['Image']:
        #load images into images of size 100x100x3
        img = image.load_img(datadir+"/"+dataset+"/"+fig, target_size=(100, 100, 3))
        x = image.img_to_array(img)
        #x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        x = x.astype('float32')/255
        X_train[count] = x
        if (count%500 == 0):
            print("Processing image: ", count+1, ", ", fig)
        count += 1
    
    return X_train

In [ ]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    # print(integer_encoded)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    # print(onehot_encoded)

    y = onehot_encoded
    # print(y.shape)
    return y, label_encoder

In [ ]:
X = prepareImages(train_df, train_df.shape[0], "train")
#X /= 100
X.shape

In [ ]:
y, label_encoder = prepare_labels(train_df['Id'])


In [ ]:
y.shape
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(X.shape)

In [ ]:
def CNN_Simple(): 
    model = Sequential()

    model.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (100, 100, 3)))

    model.add(BatchNormalization(axis = 3, name = 'bn0'))
    model.add(Activation('relu'))

    model.add(MaxPooling2D((2, 2), name='max_pool'))
    model.add(Conv2D(64, (3, 3), strides = (1,1), name="conv1"))
    model.add(Activation('relu'))
    model.add(AveragePooling2D((3, 3), name='avg_pool'))

    model.add(Flatten())
    model.add(Dense(500, activation="relu", name='rl'))
    model.add(Dropout(0.8))
    model.add(Dense(y.shape[1], activation='softmax', name='sm'))

    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    model.summary()
    
    return model

### Transfer Learning

In [ ]:
bottleneck_features = np.load(pretrainedmodeldir+"/DogResnet50Data.npz")
train_Resnet50 = bottleneck_features['train']
test_Resnet50 = bottleneck_features['test']

def resnet_model(trainbottleneckfeatures):
    model = Sequential()
    
    model.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (100, 100, 3)))

    model.add(GlobalAveragePooling2D(input_shape=trainbottleneckfeatures.shape[1:]))
    model.add(Dense(500, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(y.shape[1], activation='softmax', name='sm'))
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
from keras.applications import MobileNet

def mobilenet_model():
    base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

    xx=base_model.output
    xx=GlobalAveragePooling2D()(xx)
    xx=Dense(1024,activation='relu')(xx) #we add dense layers so that the model can learn more complex functions and classify for better results.
    xx=Dense(1024,activation='relu')(xx) #dense layer 2
    xx=Dense(512,activation='relu')(xx) #dense layer 3
    xx=Dropout(0.8)(xx)
    preds=Dense(y.shape[1],activation='softmax')(xx) #final layer with softmax activation
    
    model=Model(inputs=base_model.input,outputs=preds)
    for i,layer in enumerate(model.layers):
        print(i,layer.name)
    print(model.summary())
    # or if we want to set the first 20 layers of the network to be non-trainable
    for layer in model.layers[:20]:
        layer.trainable=False
    for layer in model.layers[20:]:
        layer.trainable=True
    
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model
              
              

In [ ]:
print(train_Resnet50.shape[1:])
print(X.shape)
print("Resnet Model: ")
#model = resnet_model(train_Resnet50)
print("mobile Net Model: ")
mobilenetmodel = mobilenet_model()
#model = CNN_Simple()
#history = model.fit(X_train, y_train, epochs=10, batch_size=100, verbose=1)
historymobilenet = mobilenetmodel.fit(X_train, y_train, epochs=10, batch_size=100, verbose=1)
gc.collect()

In [ ]:
plt.plot(history.history['acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

In [ ]:
predictions = model.predict(np.array(X_test), verbose=1)


In [ ]:
pred_df = copy.copy(train_df)
for i, pred in enumerate(predictions):
    pred_df.loc[i, 'Id'] = label_encoder.inverse_transform(pred.argsort()[-5:][::-1])

In [ ]:
print(pred_df.head(10))


### Evaluation of Prediction Precision

In [ ]:
def map_per_image(label, predictions):
    """Computes the precision score of one image.

    Parameters
    ----------
    label : string
            The true label of the image
    predictions : list
            A list of predicted elements (order does matter, 5 predictions allowed per image)

    Returns
    -------
    score : double
    """   
    score = 0.0
    try:
        if not len(np.where(predictions == label)[0])==0:
            score =  1.0/(float(np.where(predictions == label)[0]) + 1)
            return score
        else:
            return 0.
            
    except ValueError:
        return 0.0

def map_per_set(labels, predictions):
    """Computes the average over multiple images.

    Parameters
    ----------
    labels : list
             A list of the true labels. (Only one true label per images allowed!)
    predictions : list of list
             A list of predicted elements (order does matter, 5 predictions allowed per image)

    Returns
    -------
    score : double
    """
    return np.mean([map_per_image(l, p) for l,p in zip(labels, predictions)])


In [ ]:
totalaccuracy = 0.
totalaccuracy = map_per_set(train_df['Id'],pred_df['Id'])

In [ ]:
print(totalaccuracy)

### Create submission.csv (not in use for Udacity Capstone project, but created anyway)

In [ ]:
pred_df.to_csv('submission.csv', index=False)

### Predict for Testdata

In [ ]:
test = os.listdir(datadir+"/test/")
print(len(test))

In [ ]:
col = ['Image']
test_df = pd.DataFrame(test, columns=col)
test_df['Id'] = ''

In [ ]:
X = prepareImages(test_df, test_df.shape[0], "test")
X /= 255

In [ ]:
predictions = model.predict(np.array(X), verbose=1)


In [ ]:
for i, pred in enumerate(predictions):
    test_df.loc[i, 'Id'] = ' '.join(label_encoder.inverse_transform(pred.argsort()[-5:][::-1]))

In [ ]:
test_df.head(10)
test_df.to_csv('submission.csv', index=False)